In [1]:
import duckdb
import numpy as np
import pandas as pd


In [2]:
con = duckdb.connect("/Users/lilou-dangthai/HOMEWARE/db/openflights.duckdb")

con.sql("SHOW TABLES").show()


┌───────────────┐
│     name      │
│    varchar    │
├───────────────┤
│ airlines      │
│ airports      │
│ countries     │
│ dim_airlines  │
│ dim_airports  │
│ fct_routes    │
│ planes        │
│ routes        │
│ stg_airlines  │
│ stg_airports  │
│ stg_countries │
│ stg_planes    │
│ stg_routes    │
├───────────────┤
│    13 rows    │
└───────────────┘



In [3]:
df_airports = con.sql("SELECT * FROM stg_airports").df()
df_airlines = con.sql("SELECT * FROM stg_airlines").df()
df_countries = con.sql("SELECT * FROM stg_countries").df()
df_planes = con.sql("SELECT * FROM stg_planes").df()    
df_routes = con.sql("SELECT * FROM stg_routes").df()
df_fct_route = con.sql("SELECT * FROM fct_routes").df()

In [4]:
df_routes

,Airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,Equipment
0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,2B,410,CEK,2968,OVB,4078,None,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3
67659,ZM,19016,DME,4029,FRU,2912,None,0,734
67660,ZM,19016,FRU,2912,DME,4029,None,0,734
67661,ZM,19016,FRU,2912,OSS,2913,None,0,734


In [5]:
def convert_datatypes(df, column_types):
    for column, dtype in column_types.items():
        if dtype == 'Int32':
            df[column] = pd.to_numeric(df[column], errors='coerce').astype('Int32')
        elif dtype == 'int32':
            df[column] = pd.to_numeric(df[column], errors='coerce').fillna(-1).astype('int32')
        else:
            df[column] = df[column].astype(dtype)
    return df

In [6]:
convert_datatypes(df_routes, {'source_airport_id': 'Int32', 'destination_airport_id': 'Int32'})

,Airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,Equipment
0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,2B,410,CEK,2968,OVB,4078,None,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3
67659,ZM,19016,DME,4029,FRU,2912,None,0,734
67660,ZM,19016,FRU,2912,DME,4029,None,0,734
67661,ZM,19016,FRU,2912,OSS,2913,None,0,734


In [7]:
# convert to pandas nullable Int32 (preserves missing values)
df_routes['airline_id'] = pd.to_numeric(df_routes['airline_id'], errors='coerce').astype('Int32')
df_routes.dtypes

# If you really need numpy int32 (replace missing with -1):
# df_routes['airline_id'] = pd.to_numeric(df_routes['airline_id'], errors='coerce').fillna(-1).astype('int32')

Airline                   object
airline_id                 Int32
source_airport            object
source_airport_id          Int32
destination_airport       object
destination_airport_id     Int32
codeshare                 object
stops                      int32
Equipment                 object
dtype: object

In [14]:
pd.merge(df_routes, df_airlines, on="airline_id", how="left") 

,Airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,equipment,airline_name,Alias,IATA,ICAO,Callsign,Country,Active
0,2B,410,AER,2965,KZN,2990,None,0,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
1,2B,410,ASF,2966,KZN,2990,None,0,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
2,2B,410,ASF,2966,MRV,2962,None,0,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
3,2B,410,CEK,2968,KZN,2990,None,0,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
4,2B,410,CEK,2968,OVB,4078,None,0,Aerocondor,\N,2B,ARD,AEROCONDOR,Portugal,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,None,0,Regional Express,\N,ZL,RXA,REX,Australia,Y
67659,ZM,19016,DME,4029,FRU,2912,None,0,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
67660,ZM,19016,FRU,2912,DME,4029,None,0,Apache Air,Apache,ZM,IWA,APACHE,United States,Y
67661,ZM,19016,FRU,2912,OSS,2913,None,0,Apache Air,Apache,ZM,IWA,APACHE,United States,Y


In [22]:
df_airports

,airport_id,airport_name,City,Country,IATA,ICAO,Latitude,Longitude
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001
...,...,...,...,...,...,...,...,...
7693,14106,Rogachyovo Air Base,Belaya,Russia,\N,ULDA,71.616699,52.478298
7694,14107,Ulan-Ude East Airport,Ulan Ude,Russia,\N,XIUW,51.849998,107.737999
7695,14108,Krechevitsy Air Base,Novgorod,Russia,\N,ULLK,58.625000,31.385000
7696,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198


In [24]:
df_merged = df_routes.copy()
df_merged = df_merged.merge(df_airlines, on="airline_id", how="left")
df_merged = df_merged.merge(df_airports, left_on="source_airport_id", right_on="airport_id", how="left", suffixes=('', '_source'))
df_merged = df_merged.merge(df_airports, left_on="destination_airport_id", right_on="airport_id")


In [26]:
df_merged.columns

Index(['Airline', 'airline_id', 'source_airport', 'source_airport_id',
       'destination_airport', 'destination_airport_id', 'codeshare',
       'equipment', 'airline_name', 'Alias', 'IATA_x', 'ICAO_x', 'Callsign',
       'Country_x', 'Active', 'airport_id_x', 'airport_name_x', 'City_x',
       'Country_source', 'IATA_source', 'ICAO_source', 'Latitude_x',
       'Longitude_x', 'airport_id_y', 'airport_name_y', 'City_y', 'Country_y',
       'IATA_y', 'ICAO_y', 'Latitude_y', 'Longitude_y'],
      dtype='object')

In [29]:
df_merged.head()

,Airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,equipment,airline_name,Alias,...,Latitude_x,Longitude_x,airport_id_y,airport_name_y,City_y,Country_y,IATA_y,ICAO_y,Latitude_y,Longitude_y
0,2B,410,AER,2965,KZN,2990,None,0,Aerocondor,\N,...,43.449902,39.956600,2990,Kazan International Airport,Kazan,Russia,KZN,UWKD,55.606201,49.278702
1,2B,410,ASF,2966,KZN,2990,None,0,Aerocondor,\N,...,46.283298,48.006302,2990,Kazan International Airport,Kazan,Russia,KZN,UWKD,55.606201,49.278702
2,2B,410,ASF,2966,MRV,2962,None,0,Aerocondor,\N,...,46.283298,48.006302,2962,Mineralnyye Vody Airport,Mineralnye Vody,Russia,MRV,URMM,44.225101,43.081902
3,2B,410,CEK,2968,KZN,2990,None,0,Aerocondor,\N,...,55.305801,61.503300,2990,Kazan International Airport,Kazan,Russia,KZN,UWKD,55.606201,49.278702
4,2B,410,CEK,2968,OVB,4078,None,0,Aerocondor,\N,...,55.305801,61.503300,4078,Tolmachevo Airport,Novosibirsk,Russia,OVB,UNNT,55.012600,82.650703


In [6]:
df_airline_routes = df_merged[
    ['airline_id', 'airline_name',
     'airport_name_x', 'City_x', 'Country_source',
     'airport_name_y', 'City_y', 'Country_y']
].rename(columns={
    'airport_name_x': 'source_airport_name',
    'City_x': 'source_city',
    'Country_source': 'source_country',
    'airport_name_y': 'destination_airport_name',
    'City_y': 'destination_city',
    'Country_y': 'destination_country'
})
df_airline_routes.head()

NameError: name 'df_merged' is not defined

In [30]:
df_merged[df_merged['airline_id','airline_name', 'Country_source', 'city_x', 'country_y', 'city_y']]

KeyError: ('airline_id', 'airline_name', 'Country_source', 'city_x', 'country_y', 'city_y')

In [8]:
df_fct_route

,airline_id,airline_name,source_airport_id,source_airport_name,source_country,destination_airport_id,destination_airport_name,destination_country,stops,Equipment,is_direct
0,410,Aerocondor,2965,Sochi International Airport,Russia,2990,Kazan International Airport,Russia,0,CR2,True
1,410,Aerocondor,2966,Astrakhan Airport,Russia,2990,Kazan International Airport,Russia,0,CR2,True
2,410,Aerocondor,2966,Astrakhan Airport,Russia,2962,Mineralnyye Vody Airport,Russia,0,CR2,True
3,410,Aerocondor,2968,Chelyabinsk Balandino Airport,Russia,2990,Kazan International Airport,Russia,0,CR2,True
4,410,Aerocondor,2968,Chelyabinsk Balandino Airport,Russia,4078,Tolmachevo Airport,Russia,0,CR2,True
...,...,...,...,...,...,...,...,...,...,...,...
67235,3498,Midwest Airlines (Egypt),9756,None,None,9757,None,None,0,CNC,True
67236,3498,Midwest Airlines (Egypt),9756,None,None,7288,None,None,0,CNC,True
67237,3498,Midwest Airlines (Egypt),6463,None,None,7287,None,None,0,CNC,True
67238,1266,Avient Aviation,7145,None,None,7147,None,None,0,DHP,True


In [11]:
df_routes[df_routes['source_airport_id'].isna()]

,Airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,Equipment
38,2B,410,TGK,<NA>,DME,4029,None,0,CR2
54,2G,1654,KCK,<NA>,IKT,2937,None,0,AN4
1163,4O,17885,PQM,<NA>,MEX,1824,None,0,SU9
2113,5Z,18946,PBZ,<NA>,CPT,797,None,0,BEH
2114,5Z,18946,PBZ,<NA>,JNB,813,None,0,BEH
...,...,...,...,...,...,...,...,...,...
66353,Z8,603,ORU,<NA>,SRZ,4247,None,0,SWM
67468,ZI,21,MLH,<NA>,ALG,210,None,0,320
67469,ZI,21,MLH,<NA>,CZL,221,None,0,320
67470,ZI,21,MLH,<NA>,ORN,231,None,0,319


In [12]:
df_fct_route[df_fct_route['source_airport_id'].isna()]

,airline_id,airline_name,source_airport_id,source_airport_name,source_country,destination_airport_id,destination_airport_name,destination_country,stops,Equipment,is_direct
3867,18946,VivaColombia,<NA>,None,None,797,Cape Town International Airport,South Africa,0,BEH,True
3868,18946,VivaColombia,<NA>,None,None,813,OR Tambo International Airport,South Africa,0,BEH,True
3869,2850,IndiGo Airlines,<NA>,None,None,2994,Sardar Vallabhbhai Patel International Airport,India,0,320,True
3870,2850,IndiGo Airlines,<NA>,None,None,3042,Biju Patnaik Airport,India,0,320,True
3871,2850,IndiGo Airlines,<NA>,None,None,3131,Kempegowda International Airport,India,0,320,True
...,...,...,...,...,...,...,...,...,...,...,...
67647,3498,Midwest Airlines (Egypt),<NA>,None,None,7029,None,None,0,CNC,True
67657,692,Air Tahiti,<NA>,None,None,<NA>,None,None,0,ATR,True
67658,692,Air Tahiti,<NA>,None,None,<NA>,None,None,0,ATR,True
67659,5461,Wizz Air,<NA>,None,None,<NA>,None,None,0,320,True
